In [ ]:
# Check core SDK version number
import azureml.core
print("Core version:", azureml.core.VERSION)

#### Connect to workspace via config.json

In [ ]:
from azureml.core import Workspace
ws = Workspace.from_config()
ws

#### Import required modules

In [ ]:
from azureml.core import Workspace, Datastore, Dataset
from azureml.data.datapath import DataPath
import azureml.dataprep

# Uploading to Regular Datastore

## Register Azure Data Lake Storage Gen1 (ADLS Gen1) as Datastore

For ADLS Gen1 and ADLS Gen2, you will need Service Principal (SP) to access data. Service principals need to be assigned proper RBAC roles for enabling access to data in the storage account. For example, for ADLS Gen2, you will need to assign Storage Blob Data Contributor/Owner roles to the SP. 

[How to use portal to create Azure AD service principal.](https://docs.microsoft.com/en-us/azure/active-directory/develop/howto-create-service-principal-portal)

In [ ]:
adlsgen1_datastore_name ='adlgen1store' # Datastore name
store_name = '<your ADLS store name>' # ADLS Gen 1 storage account name
subscription_id = '<your subscription id>'
resource_group = '<your resource group>' # resource group for the ADLS Gen1 storage account
tenant_id = '<your tenant id>' # tenant id for the service principal
client_id = '<your SP client id>' # client id for the service principal
client_secret = '<your SP client secret>' # the secret of service principal

adls_datastore = Datastore.register_azure_data_lake(
   workspace=ws,
   datastore_name=adlsgen1_datastore_name,
   subscription_id=subscription_id, # subscription for the ADLS Gen1 storage account
   resource_group=resource_group, # resource group of ADLS Gen1 storage account
   store_name=store_name, # ADLS Gen1 storage account name
   tenant_id=tenant_id, # tenant id for the service principal
   client_id=client_id, # client id for the service principal
   client_secret=client_secret) # the secret of service principal

## Upload files to datastore and create a file dataset

```python
    def upload_directory(src_dir, target, pattern=None, overwrite=False, show_progress=True):
        """Upload source directory to target datastore and create a file dataset

        :param src_dir: The local directory to upload.
        :type src_dir: str
        :param target: Required, the datastore path where the files will be uploaded to.
        :type target: azureml.data.datapath.DataPath, azureml.core.datastore.Datastore
            or tuple(azureml.core.datastore.Datastore, str) object
        :param pattern: Optional, If provided, will filter all the path names matching the given pattern,
            similar to Python glob package, supporting '*', '?', and character ranges expressed with [].
        :type pattern: str
        :param show_progress: Optional,
            indicates whether to show progress of the upload in the console. Defaults to be True.
        :type show_progress: bool
        :return: The created file dataset.
        :rtype: azureml.data.FileDataset
        """
```

<font color='red' size=2>
Please make sure both your account and the Service Pricipal used for the Azure Data Lake Store Gen1 are granted adequate permissions to access the folder in the data lake.
</font>

In [ ]:
ds1 = Dataset.File.upload_directory(src_dir='<your source folder for uploading>/',
           target=DataPath(adls_datastore, '<upload path on the datastore>'),
           show_progress=True)

In [ ]:
# download files back to local
ds1.download("./downloads")

In [ ]:
# upload source files based on their path name using Pattern

ds2 = Dataset.File.upload_directory(src_dir='<your source folder for uploading>/',
           target=DataPath(adls_datastore, '<upload path on the datastore>'),
           pattern='<source files filtering pattern>',
           show_progress=True)

# Uploading to credential-less datastore

## Register Azure Data Lake Storage Gen1 (ADLS Gen1) as credential-less datastore

In [ ]:
adlsgen1_datastore_name = 'adlgen1credlessstore' # Datastore name
store_name = '<your ADLS store name>' # ADLS Gen1 storage account name
subscription_id = '<your subscription id>' # subscription for the ADLS Gen1 storage account
resource_group = '<your resource group>' # resource group for the ADLS Gen1 storage account

adls_datastore_cred_less = Datastore.register_azure_data_lake(
   workspace=ws,
   datastore_name=adlsgen1_datastore_name, # Datastore name
   subscription_id=subscription_id, # subscription id for the ADLS Gen1 storage account
   resource_group=resource_group, # resource group for the ADLS Gen1 storage account
   store_name=store_name) # ADLS Gen1 storage account name

## Upload files to cred-less datastore and create a file dataset


In [ ]:
ds3 = Dataset.File.upload_directory(src_dir='<your source folder for uploading>/',
           target=DataPath(adls_datastore_cred_less, '<upload path on the datastore>'),
           overwrite=True,
           show_progress=True)

# Uploading to credential-less datastore using MSI

## Create or Attach existing compute resource

<font color='red' size=2>
If using an existing compute target, please make sure the compute target has an identity attached, or type a new name to let the below script create a new one.
</font>

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = '<your compute cluster name>'

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', 
                                                           max_nodes=4,
                                                           identity_type='SystemAssigned')

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it uses the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

<font color='red' size=2>
    Make sure that you grant access to the compute cluster created here in the Azure Data Lake Store Gen1 account:<br>    
    (Not only the storage account, but also the specific folder to access)<br>    
    <dd>&emsp;1. Go to the Azure Data Lake Store Gen1 portal, click on "Data explorer" </dd>
    <dd>&emsp;2. Click on "Access" then click "+", </dd>
    <dd>&emsp;3. In the opened page, click "Select" and search for below content:<br>
        &emsp;&emsp;&emsp;"{your workspace name}/computes/{the compute cluster name created above}" </dd>
    <dd>&emsp;4. Grant all needed permissions.</dd>
</font>

## Prepare the remote run script

<font color='red' size=2>
    Please replace credential-less datastore name created above and the datapath where files are uploaded.
</font>

In [ ]:
%%writefile data/run.py
def run():
    from azureml.core import Dataset, Datastore
    from azureml.data.datapath import DataPath
    from azureml.core.run import Run
    import glob

    ws = Run.get_context().experiment.workspace
    print('Got workspace')
    print(ws)
    print('Getting datastore')
    dstore = Datastore.get(ws, 'adlgen1credlessstore')
    print('Got datastore')
    print(dstore)
    datapath = DataPath(dstore, '/%s/' % 'dataset_from_compute')
    print(datapath)
    saved_dataset = Dataset.File.upload_directory(
        src_dir='./',
        target=datapath,
        overwrite=True,
        show_progress=True)
    print(saved_dataset)

run()

## Upload files from remote context to target credential-less datastore

In [ ]:
from azureml.core import ScriptRunConfig, RunConfiguration, Experiment

rc = RunConfiguration()
rc.target = compute_target

# create or load an experiment
experiment = Experiment(workspace=ws, name='MyUploadingExperiment')
# run a trial from the train.py code in your current directory
config = ScriptRunConfig(source_directory='data', script='run.py', run_config=rc)
run = experiment.submit(config)
run.wait_for_completion()